In [1]:
# 단순 (1차원) 확산 의사 코드

# 초기 조건을 생성한다.

# ... 확산 방정식 부분 읽기만

In [2]:
from array import array
import numpy

def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v * v
    return norm
    
vector = list(range(1_000_000))
%timeit norm_square_list(vector)


71.8 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
def norm_square_numpy(vector):
    return numpy.sum(vector * vector)

vector = numpy.arange(1_000_000)
%timeit norm_square_numpy(vector)

2 ms ± 61.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
def norm_square_numpy_dot(vector):
    return numpy.dot(vector, vector)

vector = numpy.arange(1_000_000)
%timeit norm_square_numpy_dot(vector)

541 µs ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
# 6.4.1 메모리 할당과 제자리 연산
# 제자리 연산이란.. +=, *= 처럼 입력 중 하나가 위치한 메모리에 결괏값을 다시 저장하는 연산

import numpy as np

In [6]:
array1 = np.random.random((10,10))
array2 = np.random.random((10,10))

In [7]:
id(array1)

2711724358896

In [8]:
id(array2)

2711724358512

In [9]:
array1 += array2
id(array1)

2711724358896

In [10]:
array1 = array1 + array2
id(array1)

2711724770960

In [11]:
# 제자리 연산을 사용하지 않으면 예상한 대로 실행 속도가 느려짐을 확인할수 있다.
# 배열이 크기가 커질수록 메모리 할당에 시간이 더 오래 걸리므로 이 차이는 더 벌이진다.
# 하지만 이 효과는 배열 크기가 CPU 캐시보다 큰 경우에만 발생한다는 점을 알아둬야한다.
# 배열이 더 작아서 입력과 출력이 모두 캐시에 들어갈 수 있다면 백터화의 이점을 누릴 수 있으므로 제자리 연산이 아닌 쪽이 더 빠르다.

In [12]:
%%timeit array1, array2 = np.random.random((2, 1000, 1000))
array1 = array1 + array2

2.91 ms ± 42.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit array1, array2 = np.random.random((2, 1000, 1000))
array1 += array2

766 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit array1, array2 = np.random.random((2, 50, 50))
array1 = array1 + array2

1.87 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [17]:
%%timeit array1, array2 = np.random.random((2, 50, 50))
array1 += array2

954 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
from sklearn.linear_model import LinearRegression

def ols_sklearn(row):
    """Solve OLS using scikit-learn's LinearRegression"""
    est = LinearRegression() 
    X = np.arange(row.shape[0]).reshape(-1, 1) # shape (14, 1)
    # note that the intercept is built inside LinearRegression
    est.fit(X, row.values) 
    m = est.coef_[0] # note c is in est.intercept_
    return m

def ols_lstsq(row):
    """Solve OLS using numpy.linalg.lstsq"""
    # build X values for [0, 13]
    X = np.arange(row.shape[0]) # shape (14,)
    ones = np.ones(row.shape[0]) # constant used to build intercept
    A = np.vstack((X, ones)).T # shape(14, 2)
    # lstsq returns the coefficient and intercept as the first result 
    # followed by the residuals and other items
    m, c = np.linalg.lstsq(A, row.values, rcond=-1)[0] 
    return m

def ols_lstsq_raw(row):
    """Variant of `ols_lstsq` where row is a numpy array (not a Series)"""
    X = np.arange(row.shape[0])
    ones = np.ones(row.shape[0])
    A = np.vstack((X, ones)).T
    m, c = np.linalg.lstsq(A, row, rcond=-1)[0] 
    return m

ModuleNotFoundError: No module named 'sklearn'

In [21]:
# 나쁜코드
import pandas as pd 

results = None
for row_idx in range(df.shape[0]):
    row = df.iloc[row_idx]
    m = ols_lstsq(row)
    
    if results is None:
        results = pd.Series([m])
    else:
        results = pd.concat((results, pd.Series([m])))

NameError: name 'df' is not defined

In [23]:
import bottleneck
import numexpr